In [5]:
import urllib
import json

In [153]:
class office_data:
    ask = "https://declarator.org/api/v1/search/sections/?office="
    
    def __init__(self, data, f):
        self.data = data
        self.family = f
        
#     def find_children(self):
#         return []

#     def __init__(self, office_id):
#         self.office_id = office_id
#         data_struct = {} 
#         data = {'next':self.ask+str(office_id)}
#         while data["next"]:
#             with urllib.request.urlopen(data["next"]) as url:
#                 data = json.loads(url.read().decode())
#             for i in data["results"]:
#                 if i["main"]["year"] not in data_struct:
#                     data_struct[i["main"]["year"]] = []
#                 data_struct[i["main"]["year"]].append(i)
#             print(data["next"])
#         #надо обойти всех child и вызвать одну из функций sort
#         child_list = self.find_children()
#         for i in child_list:
#             child_data = get_data_office(i)
#             for j in child_data:
#                 if j not in data_struct:
#                     data_struct[j] = []
#                 data_struct[j] += child_data[j]

#         self.data = data_struct # данные всех деклараций по годам
#         self.family = 0 # учитывать ли семью
        
    def true_avg(self, typ):
        years = self.data.keys()
        res = [None]  * len(years)
        for i, year in enumerate(years):
            curr_sum = 0
            curr_amount = 0
            for declaration in self.data[year]:
                for pep in declaration[typ]:
                    inc = pep['size']
                    if self.family or (pep['relative'] == None):
                        if inc != None:
                            curr_sum += inc
                curr_amount += 1
            if curr_amount != 0:
                res[i] = curr_sum / curr_amount
        return res
        
    def gender_avg(self, typ):
        years = self.data.keys()
        males = [None] * len(years)
        females = [None] * len(years)
        for i, year in enumerate(years):
            male_sum = 0
            female_sum = 0
            male_amount = 0
            female_amount = 0
            for declaration in self.data[year]:
                gender = declaration['main']['person']['gender']
                for pep in declaration[typ]:
                    inc = pep['size']
                    if self.family or (pep['relative'] == None):
                        if inc != None:
                            if gender == 'M':
                                male_sum += inc
                            elif gender == 'F':
                                female_sum += inc
                if gender == 'M':
                    male_amount += 1
                elif gender == 'F':
                    female_amount += 1
            if male_amount != 0:
                males[i] = male_sum / male_amount
            if female_amount != 0:
                females[i] = female_sum / female_amount
        
        return males, females
        
    def party_avg(self, typ):
        parties = set()
        years = self.data.keys()
        for year in years:
            for declaration in self.data[year]:
                party = declaration['main']['party']
                if party != party or party == None:
                    parties.add('Нет Данных')
                else:
                    parties.add(party['name'])
        parties = list(parties)
        res = {par : [None] * len(years) for par in parties}
        for i, year in enumerate(years):
            sums = {par : 0 for par in parties}
            amounts = {par : 0 for par in parties}
            for declaration in self.data[year]:
                party = declaration['main']['party']
                if party != party or party == None:
                    party = 'Нет Данных'
                else:
                    party = party['name']
                for pep in declaration[typ]:
                    inc = pep['size']
                    if self.family or (pep['relative'] == None):
                        if inc != None:
                            sums[party] += inc
                amounts[party] += 1
            for party in parties:
                if amounts[party] != 0:
                    res[party][i] = sums[party] / amounts[party]
        return parties, list(res.values())
    #self.data - список по годам деклараций людей

In [160]:
office = office_data(data, False)

In [161]:
office.party_avg("incomes")

(['Единая Россия', 'Нет Данных'],
 [[975657.0,
   1063445.0,
   1215654.67,
   1397083.65,
   1769658.0,
   2213239.15,
   3031778.9,
   None,
   None,
   None,
   None],
  [None,
   None,
   None,
   None,
   None,
   None,
   None,
   2769537.14,
   1567386.81,
   1478636.39,
   1445398.08]])

In [162]:
office = office_data(data, True)

In [163]:
office.party_avg('incomes')

(['Единая Россия', 'Нет Данных'],
 [[1102094.0,
   1210288.0,
   1511234.16,
   1710425.38,
   2109819.0,
   2730143.1799999997,
   3608056.82,
   None,
   None,
   None,
   None],
  [None,
   None,
   None,
   None,
   None,
   None,
   None,
   3396191.98,
   2281861.84,
   1632852.5799999998,
   1503484.85]])

In [71]:
data[2009]

[{'bonds': [],
  'incomes': [{'comment': '', 'relative': None, 'size': 1071362.0},
   {'comment': '', 'relative': {'id': 2, 'name': 'Супруг(а)'}, 'size': 131.0}],
  'main': {'document_type': {'id': 1, 'name': 'Антикоррупционная декларация'},
   'office': {'id': 10,
    'name': 'Прокуратура Республики Алтай',
    'region': {'id': 6, 'name': 'Республика Алтай'},
    'type': {'id': 18, 'name': 'Региональный, без структуры'},
    'url': 'http://www.prokuratura.gorny.ru/'},
   'party': None,
   'person': {'family_name': 'Иванов',
    'gender': 'M',
    'given_name': 'Николай',
    'id': 19812,
    'name': 'Иванов Николай Алексеевич',
    'patronymic_name': 'Алексеевич'},
   'year': 2009},
  'real_estates': [{'comment': '',
    'country': 'Россия',
    'name': '',
    'own_type': {'id': 20, 'name': 'Индивидуальная'},
    'region': None,
    'relative': None,
    'share': None,
    'square': 1400.0,
    'type': {'id': 1, 'name': 'Земельный участок'}},
   {'comment': '',
    'country': 'Россия

In [115]:
data = office.data

In [134]:
data[2010][0]['incomes']

[{'comment': '', 'relative': None, 'size': 1215654.67},
 {'comment': '',
  'relative': {'id': 2, 'name': 'Супруг(а)'},
  'size': 295579.49}]